In [2]:
import time
from bs4 import BeautifulSoup
import pandas as pd
import requests
from requests.exceptions import Timeout
import json

ITEMS_COUNT = 100

In [56]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    
}

timeout_settings = (60, 60)

In [57]:
middle_category_nums = ['367', '596', '2074', '595', '980', '981', 
                        '1477', '657', '1004', '2080', '1003', '1002',
                        '2224']

In [58]:
def get_response(url, headers):

    # GET 요청
    response = requests.get(url, headers=headers, timeout=timeout_settings)
    soup = BeautifulSoup(response.text, 'lxml')
    time.sleep(0.5)

    return soup


In [66]:
def get_products_desc(goodsNo):
    url = f'https://www.thehandsome.com/ko/PM/productDetail/{goodsNo}'

    soup = get_response(url, headers)
    
    # 상품 설명
    try:
        productsDesc = soup.select('div.prd-desc-box')[0].text
    except:
        productsDesc = None

    # 피팅 정보
    try:
        fittingInfo = soup.select('p.cmp-font')[0].text
    except:
        fittingInfo = None

    # 추가 정보
    extraInfo = []
    try:
        extra = soup.select('ul.cmp-list.list-dotType2.bottom6')
        for info in extra:
            extraInfo.append(info.text)
    except:
        pass

    return productsDesc, fittingInfo, extraInfo

In [60]:
def get_item_info(goods):
    # 상품 id
    goodsNo = goods['goodsNo']

    # 브랜드 명
    brandNm = goods['brandNm']

    # 정가
    norPrc = goods['norPrc']

    # 할인가
    salePrc = goods['salePrc']

    # 이미지    
    image_urls = []      
    try:    
        for i in range(len(goods['colorInfo'])):
            for j in range(len(goods['colorInfo'][i]['colorContInfo'])):
                colorContInfo = goods['colorInfo'][i]['colorContInfo'][j]['dispGoodsContUrl']
                image_urls.append(colorContInfo)
    except:
        pass

    # 색상 정보
    colors = []
    try:
        for color in goods['colorInfo']:
            colors.append(color['optnNm'])
    except:
        pass

    # 사이즈 정보
    sizes = []
    for i in range(len(goods['colorInfo'][0]['colorSizeInfo'])):
        size_info = goods['colorInfo'][0]['colorSizeInfo'][i]['erpSzCd']
        sizes.append(size_info)

    # 리뷰 갯수
    goodsRevCnt = goods['goodsRevCnt']

    # 상품 설명, 피팅정보, 추가정보
    productsDesc, fittingInfo, extraInfo = get_products_desc(goodsNo)

    data = {
        'goodsNo': goodsNo,
        'brandNm': brandNm,
        'norPrce': norPrc,
        'salePrc': salePrc,
        'image_urls': image_urls,
        'colors': colors,
        'sizes': sizes,
        'goodsRevCnt': goodsRevCnt,
        'productsDesc': productsDesc,
        'fittingInfo': fittingInfo,
        'extraInfo': extraInfo

    }

    return data

In [61]:
def get_items(num):
    url = f'https://www.thehandsome.com/api/display/1/ko/category/categoryGoodsList?dispMediaCd=10&sortGbn=20&pageSize={ITEMS_COUNT}&pageNo=1&norOutletGbCd=J&dispCtgNo={num}&productListLayout=4&theditedYn=N'

    goods_info = []

    soup = get_response(url, headers)
    info = soup.string
    goods_in_page = json.loads(info)['payload']['goodsList']

    for goods in goods_in_page:
        
        goods_data = get_item_info(goods)
        goods_info.append(goods_data)
        print(len(goods_info), goods_data)


    return goods_info

In [62]:
def get_one_review(review):
    
    # 상품 id
    goodsNo = review['goodsNo']

    # 내용
    revCont = review['revCont']

    # 키
    try:
        height = review['revPrfleList'][0]['mbrPrfleValNm']
    except:
        height = None
    
    # 평소 사이즈
    try:
        nor_size = review['revPrfleList'][1]['mbrPrfleValNm']
    except:
        nor_size = None

    # 체형 정보
    form_info = {'height': height,
                 'nor_size': nor_size}
    
    # 별 수
    revScrVal = review['revScrVal']

    # 리뷰 작성일
    revWrtDtm = review['revWrtDtm']

    # 유저 아이디
    loginId = review['loginId']

    # 구매 색상
    goodsClorNm = review['goodsClorNm']

    # 구매 사이즈
    goodsSzNm = review['goodsSzNm']

    # 구매 색상, 사이즈
    scu = {
        'goodsClorNm': goodsClorNm,
        'goodsSzNm': goodsSzNm
    }

    data = {
        'goodsNo': goodsNo,
        'revCont': revCont,
        'form_info': form_info,
        'scu': scu,
        'revScrVal': revScrVal,
        'revWrtDtm': revWrtDtm,
        'loginId': loginId
    }

    return data

In [63]:
def get_review_data(goodsNo, goodsRevCnt):
    url = f'https://www.thehandsome.com/api/goods/1/ko/goods/{goodsNo}/reviews?sortTypeCd=latest&revGbCd=&pageSize={goodsRevCnt}&pageNo=1'

    one_goods_reviews = []

    soup = get_response(url, headers)
    reviews = soup.string

    try:
        reviews = json.loads(reviews)['payload']['revAllList']

        for review in reviews:
            one_review = get_one_review(review)
            one_goods_reviews.append(one_review)
    except:
        print(f'{goodsNo} 상품 리뷰 에러')
    
    print(f'{goodsNo} 상품 리뷰 수: {len(one_goods_reviews)}')


    return one_goods_reviews

    

In [64]:
def main():
    all_info = []
    for num in middle_category_nums:
        middle_category_info = get_items(num)
        all_info += middle_category_info
    
    goodsNos = []

    for info in all_info:
        goodsNos.append([info['goodsNo'], info['goodsRevCnt']])

    all_reviews = []

    for goodsNo, goodsRevCnt in goodsNos:
        one_goods_reviews = get_review_data(goodsNo, goodsRevCnt)
        all_reviews += one_goods_reviews

    df_all_info = pd.DataFrame(all_info)
    df_all_reviews = pd.DataFrame(all_reviews)

    df_all_info.to_csv('../../all_info.csv', index=False)
    df_all_reviews.to_csv('../../all_reviews.csv', index=False)

In [67]:
if __name__ == '__main__':
    main()  

1 {'goodsNo': 'SY2E3WJMTE1WM2', 'brandNm': 'SYSTEM', 'norPrce': 375000, 'salePrc': 375000, 'image_urls': ['/SY/2E/SS/SY2E3WJMTE1WM2_BK_C01.jpg', '/SY/2E/SS/SY2E3WJMTE1WM2_BK_W01.jpg?rs=684X1032', '/SY/2E/SS/SY2E3WJMTE1WM2_BK_W02.jpg?rs=684X1032', '/SY/2E/SS/SY2E3WJMTE1WM2_BK_W03.jpg?rs=684X1032', '/SY/2E/SS/SY2E3WJMTE1WM2_BK_W04.jpg?rs=684X1032', '/SY/2E/SS/SY2E3WJMTE1WM2_BK_W05.jpg?rs=684X1032', '/SY/2E/SS/SY2E3WJMTE1WM2_BK_W06.jpg?rs=684X1032', '/SY/2E/SS/SY2E3WJMTE1WM2_BK_W07.jpg?rs=684X1032', '/SY/2E/SS/SY2E3WJMTE1WM2_BK_S01.jpg', '/SY/2E/SS/SY2E3WJMTE1WM2_BK_T01.jpg', '/SY/2E/SS/SY2E3WJMTE1WM2_BK_T02.jpg', '/SY/2E/SS/SY2E3WJMTE1WM2_BK_W01.jpg', '/SY/2E/SS/SY2E3WJMTE1WM2_BK_W02.jpg', '/SY/2E/SS/SY2E3WJMTE1WM2_BK_W03.jpg', '/SY/2E/SS/SY2E3WJMTE1WM2_BK_W04.jpg', '/SY/2E/SS/SY2E3WJMTE1WM2_BK_W05.jpg', '/SY/2E/SS/SY2E3WJMTE1WM2_BK_W06.jpg', '/SY/2E/SS/SY2E3WJMTE1WM2_BK_W07.jpg'], 'colors': ['BLACK'], 'sizes': ['82'], 'goodsRevCnt': 41, 'productsDesc': '은은하게 광택감이 돌며, 가볍고 바스락한 소재를 사용하여 편

In [3]:
pd.read_csv('../../all_info.csv')

,goodsNo,brandNm,norPrce,salePrc,image_urls,colors,sizes,goodsRevCnt,productsDesc,fittingInfo,extraInfo
0,SY2E3WJMTE1WM2,SYSTEM,375000,375000,"['/SY/2E/SS/SY2E3WJMTE1WM2_BK_C01.jpg', '/SY/2...",['BLACK'],['82'],41,"은은하게 광택감이 돌며, 가볍고 바스락한 소재를 사용하여 편안하면서도 스타일리시하게...","175cm, 55 사이즈 모델이 82 사이즈 착용","['앞면에 스냅 버튼 클로징', '양옆에 포켓', '소맷단에 밴딩', '밑단 내부에..."
1,TG2E3KCD049HDG,TOM GREYHOUND,134000,134000,"['/TG/2E/SS/TG2E3KCD049HDG_DN_C01.jpg', '/TG/2...","['DARK NAVY', 'IVORY']",['OS'],4,DIAGONAL은 세명의 디자이너가 설립한 간결한 라인과 실루엣에 유니크하고 페미닌...,* Model Size : 키 175cm 몸무게 44KG B 31 W 24 H 35...,"['슬림핏', '사선 버튼 라인', '골지 립 조직']"
2,TM2E4KCD261W,TIME,475000,475000,"['/TM/2E/SS/TM2E4KCD261W_BK_C01.jpg', '/TM/2E/...","['BLACK', 'OFF WHITE', 'SEPIA BEIGE']","['90', '95']",17,군더더기 없이 깔끔한 디자인으로 높은 활용도를 자랑하는 니트 가디건입니다. 간결한 ...,"175cm, 55 사이즈 모델이 90 사이즈 착용","['100% 면 소재', '브이넥 디자인', '앞면에 버튼 클로징', '하프 슬리브..."
3,CM2E4KOT531WM1,the CASHMERE,495000,495000,"['/CM/2E/SS/CM2E4KOT531WM1_CB_C01.jpg', '/CM/2...","['CHOCOLATE BROWN', 'OFF WHITE']","['85', '90']",21,볼드하게 짜여진 크로셰 니팅이 눈길을 끄는 니트 가디건입니다. 크롭한 기장이 특징이...,"173cm, 55 사이즈 모델이 85 사이즈 착용","['이탈리아 수입 원사', '면 혼방 소재', '앞면에 버튼 클로징', '하프 슬리..."
4,CM2E3WJC158WP1,the CASHMERE,695000,695000,"['/CM/2E/SS/CM2E3WJC158WP1_LE_C01.jpg', '/CM/2...","['LIGHT BEIGE', 'OFF WHITE']","['82', '88']",0,내추럴한 질감이 돋보이는 린넨 혼방 소재를 사용하여 S/S 시즌 무드가 느껴지는 싱...,"173cm, 55 사이즈 모델이 82 사이즈 착용","['린넨 혼방 소재', '앞면에 버튼 클로징', '양옆에 포켓', '소맷단에 버튼 ..."
...,...,...,...,...,...,...,...,...,...,...,...
1218,TH2DBKRN039N,TIME HOMME,415000,415000,"['/TH/2D/FW/TH2DBKRN039N_DB_C01.jpg', '/TH/2D/...","['DARK BROWN', 'MINT', 'SKY BLUE']","['95', '100', '105', '110']",13,서로 다른 컬러를 믹스한 멜란지 텍스처가 돋보이는 니트 탑으로 부드럽고 신축성 있는...,"186cm, 105 사이즈 모델이 105 사이즈 착용","['램스 울과 알파카, 스판 혼방 소재', '각 끝단 리브 마감', '라운드넥 디자인']"
1219,TH2DBQJM345N,TIME HOMME,1080000,1080000,"['/TH/2D/FW/TH2DBQJM345N_BK_C01.jpg', '/TH/2D/...","['BLACK', 'KHAKI']","['95', '100', '105']",0,목을 부드럽게 감싸주는 하이넥 스타일의 다운 점퍼로 가볍고 바스락한 소재를 사용하였...,"187cm, 100 사이즈 모델이 105 사이즈 착용","['앞면에 스냅 버튼과 지퍼 클로징', '양옆에 포켓과 내부에 1개의 인포켓', '..."
1220,TH2DBQOT343N,TIME HOMME,1080000,1080000,"['/TH/2D/FW/TH2DBQOT343N_BK_C01.jpg', '/TH/2D/...",['BLACK'],"['95', '100', '105']",7,버튼 여밈이 돋보이는 다운 점퍼로 탄탄하면서 가벼운 면 혼방 소재를 사용하였습니다....,"187cm, 100 사이즈 모델이 105 사이즈 착용","['면 혼방 소재', '앞면에 버튼과 지퍼 클로징', '앞면에 3개의 포켓과 내부에..."
1221,TH2DBWCT420N,TIME HOMME,1150000,1150000,"['/TH/2D/FW/TH2DBWCT420N_BK_C01.jpg', '/TH/2D/...","['BLACK', 'Sepia Brown']","['95', '100', '105']",16,램스 울과 캐시미어 혼방 소재를 사용하여 가볍고 포근한 터치감이 돋보이는 발마칸 스...,"186cm, 105 사이즈 모델이 105 사이즈 착용","['램스 울과 캐시미어 혼방 소재', '앞면의 히든 버튼 클로징', '드롭 숄더 디..."


In [4]:
pd.read_csv('../../all_reviews.csv')

,goodsNo,revCont,form_info,scu,revScrVal,revWrtDtm,loginId
0,SY2E3WJMTE1WM2,평소 잘 안입는 스타일인데 입어보니 너무 맘에 들어요!\n힙해보인다 해야하나..? ...,"{'height': '172cm', 'nor_size': '66'}","{'goodsClorNm': 'BLACK', 'goodsSzNm': '82'}",5,2024.04.22,cyj****
1,SY2E3WJMTE1WM2,편하게 툭 걸치기 너무 좋아요. 소재도 가볍고 사이즈도 넉넉해서 어느 옷에나 매치하...,"{'height': '162cm', 'nor_size': '55'}","{'goodsClorNm': 'BLACK', 'goodsSzNm': '82'}",5,2024.04.21,hey*****************
2,SY2E3WJMTE1WM2,간절기에 여기저기 잘어울리게 입을수있을것같아요 \n바스락 거리는 느낌이 좋네요,"{'height': '172cm', 'nor_size': '66'}","{'goodsClorNm': 'BLACK', 'goodsSzNm': '82'}",5,2024.04.20,lid******
3,SY2E3WJMTE1WM2,살까말까 고민진짜 많이했는데\n받아서 입어보니 핏도 이쁘고 \n휘뚜루마뚜루 아무데나...,"{'height': 'cm', 'nor_size': '66'}","{'goodsClorNm': 'BLACK', 'goodsSzNm': '82'}",5,2024.04.20,sso******
4,SY2E3WJMTE1WM2,좋습니다~! 마음에 들어요~~~~,"{'height': 'cm', 'nor_size': '55'}","{'goodsClorNm': 'BLACK', 'goodsSzNm': '82'}",5,2024.04.20,sjh****
...,...,...,...,...,...,...,...
9269,TH2DBWCT420N,역시 코트는 타임 \n넘 만족스럽습니다,"{'height': '173cm', 'nor_size': '95'}","{'goodsClorNm': 'BLACK', 'goodsSzNm': '95'}",5,2023.11.27,kis****
9270,TH2DBWCT420N,오버핏으로 저렴한 가격으로 잘 샀어요\n완전오버핏이네요,"{'height': 'cm', 'nor_size': '105'}","{'goodsClorNm': 'BLACK', 'goodsSzNm': '105'}",5,2023.11.25,cal******
9271,TH2DBWCT420N,가볍고 따뜻하고 좋습니다,"{'height': '180cm', 'nor_size': None}","{'goodsClorNm': 'BLACK', 'goodsSzNm': '100'}",5,2023.11.25,kty******
9272,TH2D9TTN645N,좋습니다. 잘입겠습니다.,"{'height': 'cm', 'nor_size': None}","{'goodsClorNm': 'SMOKE BLUE', 'goodsSzNm': '105'}",5,2024.01.09,roc*****
